# Test Veränderungen im Baumkataster der Stadt Zürich

Im Juli 2021 fanden europaweit grosse Überschwemmungen und Sturmschäden statt. Auch in Zürich, insbesondere in Zürich-West (Altstetten und Albisrieden), ging eine Schneise der Zerstörung und zahlreiche Bäume fielen um oder wurden abgeknickt. Siehe dazu z.B. den Bericht in der NZZ am 14.07.2021: [Der Leiter der Zürcher Grünanlagen sagt: «Sturm hat den Baumbestand mancherorts um mehr als die Hälfte reduziert.»](https://www.nzz.ch/zuerich/sturm-ueber-zuerich-teilweise-der-halbe-baumbestand-zerstoert-ld.1635541).

Das Ziel dieses Notebooks ist es, die Veränderungen des Baumkatasters generell und möglicherweise Ende 2021 auch insbesondere bezüglich der Sturmschäden zu prüfen.
Wahrscheinlich ist das nicht möglich, weil zwischen einer Anpassung in der Datenbank (Mutation), eine Fällung auf Grund des Alters oder der Gesundheit und einer Fällung auf Grund von Sturmschäden keine direkten Hinweise vermerkt sind. Aber ein Versuch ist es wert.

<!---  
Dieses Notebook verwendet als Vorlage die Webseite «[Kitas in Berlin - Jupyter Notebook](https://juanitorduz.github.io/kitas_berlin/)» 
---!>

Mit Colab kann **dieses Jupyter-Notebook interaktiv im Browser** gestartet werden:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DonGoginho/myPy_RIS_API/blob/main/RIS_API_AL_Geschaefte.ipynb)


**Inhaltsverzeichnis:**

1. [Daten importieren](#LiesDatenein)
2. [Daten prüfen](#)
3. [Daten analysieren](#Explorative_Analyse)



## Importiere notwendige Packages:

In [ ]:
#pip install geopandas fiona requests folium mplleaflet contextily seaborn datetime

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona; 
import mplleaflet
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from datetime import datetime
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

pd.options.display.float_format = '{:.0f}'.format
#pd.set_option('display.width', 100)
#pd.set_option('display.max_columns', 15)

Schreibe aktuelles Datum in Variable `date_today`
Für weitere Infos zu Datetime siehe https://www.programiz.com/python-programming/datetime/strftime

In [66]:
#Soll das aktuelle Datum beim Geopackage-Namen integriert werden oder nicht? False = nein. Generiert ein Outputfile mit der Endung _tmp, welches sich selber überschreibt
activate_date = False;

now = datetime.now() # current date and time

if activate_date == True:
    date_today = "_"+now.strftime("%Y-%m-%d")
    #store_o_gis = False
    
else:
    date_today ="_tmp"
    #store_o_gis = True

## Lies Daten ein


Das **Baumkatster der Stadt Zürich** wird seit mehreren Jahren als OGD angeboten: https://data.stadt-zuerich.ch/dataset/geo_baumkataster . 
Im Baumkataster werden alle städtischen Bäume im Strassenraum erfasst, welche von Grün Stadt Zürich verwaltet oder gepflegt werden. Seit 2019 wird das Baumkataster durch das Obstbauminventar sowie Bäume ausgewählter öffentlicher Grünanlagen und private Bäume ergänzt.

Zum Download steht es seit 2019 auch auf dem städtischen [Geodatenportal](https://www.stadt-zuerich.ch/geodaten) zur Verfügung: 

- Geodaten: https://www.stadt-zuerich.ch/geodaten/download/Baumkataster
- Metadaten: https://www.geocat.ch/geonetwork/srv/ger/md.viewer#/full_view/5b17c52e-1fcd-e0e4-47b2-ab784f1bead7

### Importiere den aktuellsten Baumbestand des Baumkatasters direkt ab Geodatenportal

Das Geoportal bietet zahlreiche Geoformate zum Download (Shapefiles, Geopackages, csv, dxf, etc.) wie auch Webservcies (WFS, WMS, WMTS) an.

Um direkt den aktuellsten Datensatz zu beziehen, bietet sich der **GEOJSON-Webservice via WFS** an. Dazu muss man noch die Layer im Datensatz kennen. Dieser ist unterteilt in:

1. `baumkataster_baumstandorte` (Baumstandorte in der Stadt Zürich)
2. `baumkataster_kronendurchmesser` (Darstellung des Kronendurchmessers für den CAD-Export.9)
3. `baumstandorte_k` (Darstellung des Bauzmstandortes als Kreis (10cm Radius) für den CAD-Export)

Uns interssiert für diese Fragestellung der 1. Layer, die `baumkataster_baumstandorte`. Die URL zum GeoJSON-Service des aktuellen Bestand lautet daher:

https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Baumkataster?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=baumkataster_baumstandorte



In [67]:
#Direkter GeoJSON-Import via WFS-GeoJSON-Service

geojson_url = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Baumkataster?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=baumkataster_baumstandorte"
df_baumkataster_baumstandorte_aktuell = gpd.read_file(geojson_url)

df_baumkataster_baumstandorte_aktuell.head(2)

,id,baumartlat,baumgattunglat,baumnamedeu,baumnamelat,baumnummer,baumtyp,baumtyptext,genauigkeit,kategorie,kronendurchmesser,objid,pflanzjahr,poi_id,quartier,status,strasse,geometry
0,baumkataster_baumstandorte.0,pseudoplatanus,Acer,"Berg-Ahorn, Wald-Ahorn",Acer pseudoplatanus,SW-607,0,nicht zugeordnet,Eingemessen,Strassenbaum,14,0,NaN,SW-607,Schwamendingen-Mitte,Strassenbaum,Ueberlandstr.,POINT (8.57845 47.40898)
1,baumkataster_baumstandorte.3,platanoides,Acer,Spitz-Ahorn,Acer platanoides,SW-612,0,nicht zugeordnet,Eingemessen,Strassenbaum,12,3,NaN,SW-612,Schwamendingen-Mitte,Strassenbaum,Ueberlandstr.,POINT (8.57783 47.40913)


Speichere das den direkter GeoJSON-Import via WFS-GeoJSON-Service als Geopackage lokal ab

In [68]:
#Import mit Geopandas, Webseite: https://geopandas.org/docs/user_guide/io.html oder  import fiona; help(fiona.open)

df_baumkataster_baumstandorte_aktuell.to_file("//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster"+date_today+".gpkg", layer='gsz_baumkataster_baumstandorte', driver="GPKG")
print("//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster"+date_today+".gpkg")

//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster_tmp.gpkg



GeoJson-Dataset zum direkten Abruf des aktuellen Bestandes: 
https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Baumkataster?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=baumkataster_baumstandorte

### Importiere frühere Baumbestände des Baumkatasters

Das Baumkataster wird meines Wissens nicht historiesiert zur Verfügung gestellt auf dem GeoServer. Ich habe bereits früher von Zeit zu Zeit Bestände abgespeichert unter 
`\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\GSZ\baumkataster`



Importiere das `Baumkataster 2021 07 18`. Beachte, dass hier das Geopackage mehrere Layers hat, aus denen einer ausgewählt werden muss. Die folgenden früheren Jahre hatten das bei den Downloadfiles noch nicht.

In [36]:
#Importiere das Baumkataster 2021 07 18
pfad_baumkataster_20210718 = r"//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster_2021_07_18.gpkg"
df_baumkataster_20210718 = gpd.read_file(pfad_baumkataster_20210718, layer='gsz_baumkataster_baumstandorte')


# df_baumkataster_20210718.info()
# Find the number of null values for all columns
#df_baumkataster_20210718.isnull().sum()

Importiere das `Baumkataster 2020`.

In [37]:
#Importiere das Baumkataster 2020
pfad_baumkataster_20201016 = r"//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster_2020_10_16.gpkg"
df_baumkataster_20201016 = gpd.read_file(pfad_baumkataster_20201016)

#df_baumkataster_20201016.info()
# Find the number of null values for all columns
#df_baumkataster_20201016.isnull().sum()

Importiere das `Baumkataster 2018`.

In [38]:
#Importiere das Baumkataster 2018
pfad_baumkataster_20181023 = r"//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster_2018_10_23.gpkg"
df_baumkataster_20181023 = gpd.read_file(pfad_baumkataster_20181023)

#df_baumkataster_20181023.info()
# Find the number of null values for all columns
#df_baumkataster_20181023.isnull().sum()

Importiere das `Baumkataster 2014`.

In [39]:
#Importiere das Baumkataster 2014
pfad_baumkataster_20141220 = r"//szh.loc/ssz/applikationen/OGD/Daten/Quelldaten/GSZ/baumkataster/baumkataster_2014_12_20.gpkg"
imp_df_baumkataster_20141220 = gpd.read_file(pfad_baumkataster_20141220)

#imp_df_baumkataster_20141220.info()
# Find the number of null values for all columns
#imp_df_baumkataster_20141220.isnull().sum()

Da das Baumkataster 2014 noch andere Attributbeschreibungen hatte, soll es in folgendem Schritt noch den aktuellen Beschreibungen angepasst werden.

In [40]:
# Rename columns bei Bedarf
rename_cols = {
    'OBJECTID': 'objid',
    'Status': 'status',
    'BaumGattun': 'baumgattunglat',
    'BaumArtLat': 'baumartlat',
    'Kategorie': 'kategorie',
    'Pflanzjahr': 'pflanzjahr',
    'Baumtyp': 'baumtyp',
    'Quartier': 'quartier',
    'Strasse': 'strasse',
    'BaumNameLa': 'baumnamelat',
    'Baumnummer': 'baumnummer',
    'POI_ID': 'poi_id',
    'BaumNameDe': 'baumnamedeu',
    'Baumtyptex': 'baumtyptext',
    'Genauigkei': 'genauigkeit',
    'geometry': 'geometry',
}

columns_to_drop = ['xtf_id']
# Format data: remove redundant columns, rename columns and add new features.

df_baumkataster_20141220 = imp_df_baumkataster_20141220 \
    .copy() \
    .drop(columns_to_drop, axis=1) \
    .rename(columns=rename_cols) \
    .assign(
        #spots_plz=lambda x: x.groupby(['district', 'plz'])['spots'].transform(np.sum)
    )


df_baumkataster_20141220.head(2)

,objid,status,baumgattunglat,baumartlat,kategorie,pflanzjahr,baumtyp,quartier,strasse,baumnamelat,baumnummer,poi_id,baumnamedeu,baumtyptext,genauigkeit,geometry
0,1,Grünanlage,nicht identifiziert,None,Parkbaum,None,0,Hirzenbach,None,nicht identifiziert,SW-10495,bm_0,nicht identifiziert,nicht zugeordnet,Bildschirmeingabe,POINT (2686741.210 1250332.560)
1,2,Strassenbaum,nicht identifiziert,None,Strassenbaum,None,0,Sihlfeld,None,nicht identifiziert,WD-7435,bm_1,nicht identifiziert,nicht zugeordnet,Bildschirmeingabe,POINT (2680433.340 1247519.140)


Da die Datenbestände aus früheren Zeiten stammen, sind sie - v.a. 2014 - noch unterschiedlich strukturiert. Falls ein Verleich zwischen 2014 und aktuell gemacht werden soll, müssen die Attributnamen angepasst werden.

In [50]:
df_baumkataster_20210718.info()
df_baumkataster_20201016.info()
df_baumkataster_20181023.info()
df_baumkataster_20141220.info()
df_baumkataster_baumstandorte_aktuell.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 74325 entries, 0 to 74324
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   objid              74325 non-null  object  
 1   poi_id             74325 non-null  object  
 2   kategorie          74325 non-null  object  
 3   quartier           74325 non-null  object  
 4   strasse            74325 non-null  object  
 5   baumgattunglat     74325 non-null  object  
 6   baumartlat         74325 non-null  object  
 7   baumnamelat        74325 non-null  object  
 8   baumnamedeu        74325 non-null  object  
 9   baumnummer         74325 non-null  object  
 10  status             74325 non-null  object  
 11  baumtyp            73686 non-null  float64 
 12  baumtyptext        74325 non-null  object  
 13  pflanzjahr         57158 non-null  float64 
 14  genauigkeit        74325 non-null  object  
 15  kronendurchmesser  74325 non-null  int64   
 

Checke welche Quartiere vorkommen

In [42]:
df_baumkataster_20210718['quartier'].unique()

array(['Schwamendingen-Mitte', 'Albisrieden', 'Hirzenbach', 'Altstetten',
       'Hard', 'Fluntern', 'Friesenberg', 'Wipkingen', 'Seebach',
       'Unterstrass', 'Oberstrass', 'Oerlikon', 'Saatlen', 'Weinegg',
       'Wollishofen', 'Hottingen', 'City', 'Enge', 'Alt-Wiedikon',
       'Leimbach', 'Seefeld', 'Witikon', 'Hirslanden', 'Sihlfeld',
       'Höngg', 'Escher Wyss', 'Mühlebach', 'Langstrasse', 'Werd',
       'Hochschulen', 'Lindenhof', 'Gewerbeschule', 'Rathaus',
       'Affoltern'], dtype=object)

In [43]:
df_baumkataster_20210718['quartier'].unique()

array(['Schwamendingen-Mitte', 'Albisrieden', 'Hirzenbach', 'Altstetten',
       'Hard', 'Fluntern', 'Friesenberg', 'Wipkingen', 'Seebach',
       'Unterstrass', 'Oberstrass', 'Oerlikon', 'Saatlen', 'Weinegg',
       'Wollishofen', 'Hottingen', 'City', 'Enge', 'Alt-Wiedikon',
       'Leimbach', 'Seefeld', 'Witikon', 'Hirslanden', 'Sihlfeld',
       'Höngg', 'Escher Wyss', 'Mühlebach', 'Langstrasse', 'Werd',
       'Hochschulen', 'Lindenhof', 'Gewerbeschule', 'Rathaus',
       'Affoltern'], dtype=object)

## Explorative_Analyse
One of the main objectives of this notebook is to do an exploratory data analysis to understand which questions this data set can answer. In addition, also determine its limitations. 

To begin, let us get the number of unique values per feature.


In [44]:
df_baumkataster_20141220.apply(lambda x: x.unique().size, axis=0)


objid             56003
status                6
baumgattunglat      110
baumartlat          327
kategorie             2
pflanzjahr           91
baumtyp               7
quartier             35
strasse            1147
baumnamelat         722
baumnummer        56003
poi_id            56003
baumnamedeu         502
baumtyptext           6
genauigkeit           7
geometry          56001
dtype: int64

In [45]:
df_baumkataster_20181023.apply(lambda x: x.unique().size, axis=0)


kategorie             2
quartier             35
strasse            8393
baumgattunglat      111
baumartlat          351
baumnamelat        1377
baumnamedeu        1109
baumnummer        66593
status                9
baumtyp               8
baumtyptext           6
pflanzjahr           94
genauigkeit           6
geometry          66547
dtype: int64

In [46]:
df_baumkataster_20210718.apply(lambda x: x.unique().size, axis=0)

objid                74325
poi_id               74325
kategorie                2
quartier                34
strasse               9194
baumgattunglat         122
baumartlat             379
baumnamelat           1482
baumnamedeu           1194
baumnummer           74325
status                   7
baumtyp                  8
baumtyptext              6
pflanzjahr              96
genauigkeit              6
kronendurchmesser       40
geometry             74323
dtype: int64

Welche Kategorien gibt es überhaupt und wie ist der Bezug zum 'Status'?

In [47]:
 #   .query('kategorie == "Parkbaum"') \
#    .agg(anz_baeume=('objid', 'count'), anz_kinder_va_sk=('objid', 'sum')) \
df_baumkataster_20210718 \
    .groupby(['kategorie','status' ]) \
    .agg(anz_baeume=('baumnummer', 'count')) \
    .sort_values('kategorie', ascending=False) \
    .head(500)

anz_baeume
kategorie    status                      
Strassenbaum Strassenbaum           20093
             Strassenbaum (A)        2490
Parkbaum     Grünanlage             38884
             Kanton                  2321
             Obst                    7927
             Schulen                   89
             Wohnsiedlungen          2521

Welche Baumart ist bei den Parkbäumen die Häufigste?

In [48]:

df_baumkataster_20210718 \
    .groupby(['kategorie','status','baumgattunglat','baumartlat','baumnamelat','baumnamedeu' ]) \
    .agg(anz_baeume=('baumnummer', 'count')) \
    .query('kategorie == "Parkbaum"') \
    .sort_values('anz_baeume', ascending=False) \
    .head(10)




anz_baeume
kategorie status     baumgattunglat baumartlat     baumnamelat         baumnamedeu                                        
Parkbaum  Grünanlage Carpinus       betulus        Carpinus betulus    Gemeine Hain- oder Weissbuche                  3130
                     Taxus          baccata        Taxus baccata       Eibe, Gewöhnliche Eibe                         2107
                     Pinus          sylvestris     Pinus sylvestris    Gewöhnlicher Kiefer, Wald-Kiefer, Föhre        1807
          Obst       Malus          domestica      Malus domestica cv. Apfel-Obstgehölz, Sorte unbekannt              1661
          Grünanlage Acer           campestre      Acer campestre      Feld-Ahorn, Hecken-Ahorn                       1639
                     Betula         pendula        Betula pendula      Sand-Birke, Weiss-Birke                        1600
                     Fraxinus       excelsior      Fraxinus excelsior  Gemeine Esche                                  1581
                     Acer           platanoides    Acer platanoides    Spitz-Ahorn                                    1514
                     Picea          abies          Picea abies         Rottanne, Fichte                               1394
                     Acer           pseudoplatanus Acer pseudoplatanus Berg-Ahorn, Wald-Ahorn                         1263

In [17]:
#    .query('kategorie == "Strassenbaum"') \
df_baumkataster_20181023 \
    .groupby(['kategorie','status','baumgattunglat','baumartlat','baumnamelat','baumnamedeu' ]) \
    .agg(anz_baeume=('baumnummer', 'count')) \
    .query('kategorie == "Parkbaum"') \
    .sort_values('anz_baeume', ascending=False) \
    .head(10)

anz_baeume
kategorie status     baumgattunglat baumartlat  baumnamelat          baumnamedeu                                        
Parkbaum  Grünanlage Carpinus       betulus     Carpinus betulus     Gemeine Hain- oder Weissbuche                  2658
                     Taxus          baccata     Taxus baccata        Eibe, Gewöhnliche Eibe                         1944
                     Fraxinus       excelsior   Fraxinus excelsior   Gemeine Esche                                  1743
          Obst       Malus          domestica   Malus domestica cv.  Apfel-Obstgehölz, Sorte unbekannt              1727
          Grünanlage Pinus          sylvestris  Pinus sylvestris     Gewöhnlicher Kiefer, Wald-Kiefer, Föhre        1478
                     Picea          abies       Picea abies          Rottanne, Fichte                               1353
                     Betula         pendula     Betula pendula       Sand-Birke, Weiss-Birke                        1343
                     Acer           campestre   Acer campestre       Feld-Ahorn, Hecken-Ahorn                       1241
                                    platanoides Acer platanoides     Spitz-Ahorn                                    1013
                     Platanus       x hispanica Platanus x hispanica Platane (occidentalis x orientalis)             947

In [51]:
#    .query('kategorie == "Strassenbaum"') \
df_baumkataster_baumstandorte_aktuell \
    .groupby(['kategorie','status','baumgattunglat','baumartlat','baumnamelat','baumnamedeu' ]) \
    .agg(anz_baeume=('baumnummer', 'count')) \
    .query('kategorie == "Parkbaum"') \
    .sort_values('anz_baeume', ascending=False) \
    .head(10)

anz_baeume
kategorie status     baumgattunglat baumartlat     baumnamelat         baumnamedeu                                        
Parkbaum  Grünanlage Carpinus       betulus        Carpinus betulus    Gemeine Hain- oder Weissbuche                  3202
                     Taxus          baccata        Taxus baccata       Eibe, Gewöhnliche Eibe                         2148
                     Pinus          sylvestris     Pinus sylvestris    Gewöhnlicher Kiefer, Wald-Kiefer, Föhre        1821
          Obst       Malus          domestica      Malus domestica cv. Apfel-Obstgehölz, Sorte unbekannt              1748
          Grünanlage Betula         pendula        Betula pendula      Sand-Birke, Weiss-Birke                        1672
                     Acer           campestre      Acer campestre      Feld-Ahorn, Hecken-Ahorn                       1660
                     Fraxinus       excelsior      Fraxinus excelsior  Gemeine Esche                                  1659
                     Acer           platanoides    Acer platanoides    Spitz-Ahorn                                    1529
                     Picea          abies          Picea abies         Rottanne, Fichte                               1460
                     Acer           pseudoplatanus Acer pseudoplatanus Berg-Ahorn, Wald-Ahorn                         1283